# АБ-тестирование

In [1]:
import pandas as pd
import numpy as np

Задача: Взять датасет из google диска: https://drive.google.com/file/d/1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_. Проверить гипотезу о том, в каком варианте теста (control/personalization) больше конверсия (converted) и значимо ли это отличие статистически.

In [2]:
#!wget 'https://drive.google.com/uc?export=download&id=1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_' -O data.zip

In [3]:
#!unzip data.zip

In [4]:
marketing_campaign = pd.read_csv('marketing_campaign.csv')
marketing_campaign

,user_id,date_served,marketing_channel,variant,language_displayed,converted
0,a1000,1/1/18,House Ads,personalization,English,True
1,a1001,1/1/18,House Ads,personalization,English,True
2,a1002,1/1/18,House Ads,personalization,English,True
3,a1003,1/1/18,House Ads,personalization,English,True
4,a1004,1/1/18,House Ads,personalization,English,True
...,...,...,...,...,...,...
10032,a11032,1/17/18,Email,control,German,True
10033,a11033,1/17/18,Email,control,German,True
10034,a11034,1/5/18,Instagram,control,German,False
10035,a11035,1/17/18,Email,control,German,True


In [5]:
marketing_campaign.variant.value_counts()

control            5091
personalization    4946
Name: variant, dtype: int64

In [6]:
marketing_campaign.converted.value_counts()

False    8946
True     1076
Name: converted, dtype: int64

Нулевая гипотеза: группы воздействия и контрольная не имеют статистически значимых отличий с учетом доверительного интервала в 95%. Проведем расчет конверсии и применим статистические методы оценки гипотез.

In [7]:
k1 = marketing_campaign.loc[(marketing_campaign['variant']=='personalization') & (marketing_campaign['converted']==True)].value_counts().shape[0]
k1

705

In [8]:
k2 = marketing_campaign.loc[(marketing_campaign['variant']=='control') & (marketing_campaign['converted']==True)].value_counts().shape[0]
k2

371

In [9]:
n1 = marketing_campaign.loc[marketing_campaign['variant']=='personalization'].value_counts().shape[0]
n1

4945

In [10]:
n2 = marketing_campaign.loc[marketing_campaign['variant']=='control'].value_counts().shape[0]
n2

5076

In [11]:
#Проанализируем конверсию по группам:

conv_pers = round(k1/n1, 2)
conv_control = round(k2/n2, 2)
    
print(f' конверсия в группе "personalization" {conv_pers}, конверсия в группе "control" {conv_control}')

 конверсия в группе "personalization" 0.14, конверсия в группе "control" 0.07


Вывод: в результате воздействия на покупателя конверсия в группе воздействия оказалась выше, чем в контрольной группе, в два раза. Это хороший показатель. Проведем проверку статистической значимости различий в данных группах.

Наиболее очевидным статистическим критерием для нашего случая выглядит z-критерий.

In [12]:
#!pip install statsmodels

In [13]:
from statsmodels.stats import proportion

z_score, z_pvalue = proportion.proportions_ztest(np.array([k1, k2]), 
                                                 np.array([n1, n2]))

print(f'Results are z_score={z_score:.3f} pvalue={z_pvalue:.3f}')

Results are z_score=11.232 pvalue=0.000


In [14]:
z_crit_value = 1.96 # соответствует доверительному интервалу в 95%

In [15]:
if z_score > z_crit_value:
    print("We may reject the null hypothesis!")
else:
    print("We have failed to reject the null hypothesis")

We may reject the null hypothesis!


Нулевая гипотеза о том, что статистически значимых различий в группах нет, отклонена. Различия статистически значимы. Мероприятия по воздействию на покупателя можно считать эффективными. Рекомендуется провести расчет финансовой обоснованности воздействия на покупателя, сопоставить расходы на воздействие с доходами от дополнительных покупок группы воздействия.